# Sparse Reconstruction Uncertainty Analysis

This notebook analyzes the uncertainty in sparse tensor tomography reconstructions by:
1. Computing a ground truth reconstruction using all available projections (240 angles)
2. Fixing a sparse subset of 60 projections
3. Repeatedly sampling 80% of these 60 projections (48 projections)
4. Computing k reconstructions and analyzing their standard deviation
5. Visualizing the uncertainty across spherical harmonic coefficients

In [ ]:
# /das/work/units/pem/p20639/envs/p20639

In [1]:
try:
    from mumott.data_handling import DataContainer
except:
    !sh ../scripts/setup.sh

INFO:Setting the number of threads to 8. If your physical cores are fewer than this number, you may want to use numba.set_num_threads(n), and os.environ["OPENBLAS_NUM_THREADS"] = f"{n}" to set the number of threads to the number of physical cores n.
INFO:Setting numba log level to WARNING.
INFO:Setting numba log level to WARNING.


In [2]:
import sys
sys.path.insert(0, '/myhome/smartt')
sys.path.insert(0, '/das/home/barbaf_l/smartTT')


import numpy as np
import matplotlib.pyplot as plt
import h5py
from pathlib import Path
from tqdm import tqdm
import copy

# Import mumott modules
from mumott.data_handling import DataContainer
from mumott.methods.basis_sets import SphericalHarmonics
from mumott.methods.projectors import SAXSProjector, SAXSProjectorCUDA
from mumott.methods.residual_calculators import GradientResidualCalculator
from mumott.optimization.loss_functions import SquaredLoss
from mumott.optimization.optimizers import LBFGS
from mumott.optimization.regularizers import Laplacian

# Import custom functions
# from smartt.data_processing import _perform_reconstruction

%matplotlib inline

In [3]:
"""
Data processing module for generating tensor tomography reconstruction datasets.

This module provides functionality to load projection data, randomly sample subsets,
perform mumott reconstructions, and save the results for machine learning training.
"""

import copy
import h5py
import numpy as np
import os
import sys
import torch
from pathlib import Path
from typing import Union, Optional, List, Tuple, Dict, Literal
from mumott.data_handling import DataContainer
from mumott.methods.basis_sets import SphericalHarmonics
from mumott.methods.projectors import SAXSProjector, SAXSProjectorCUDA
from mumott.methods.residual_calculators import GradientResidualCalculator
from mumott.optimization.loss_functions import SquaredLoss
from mumott.optimization.optimizers import LBFGS
from mumott.optimization.regularizers import Laplacian

# Import PyTorch projectors and SH forward
from smartt.projectors import forward_project, backproject, build_mumott_projector
from smartt.shutils.evaulate_sh import forward_quadrature


def _perform_reconstruction_lbfgs(
    dc: DataContainer,
    ell_max: int,
    maxiter: int,
    regularization_weight: float,
    use_cuda: bool,
    verbose: bool = False
) -> np.ndarray:
    """
    Perform a single spherical harmonic reconstruction using mumott LBFGS.
    
    Parameters
    ----------
    dc : DataContainer
        The data container with projections.
    ell_max : int
        Maximum degree for spherical harmonics expansion.
    maxiter : int
        Maximum number of iterations for the LBFGS optimizer.
    regularization_weight : float
        Weight for the Laplacian regularization term.
    use_cuda : bool
        Whether to use CUDA for computation.
    verbose : bool, default=False
        Whether to print progress information.
    
    Returns
    -------
    np.ndarray
        The reconstruction as a numpy array of shape (*volume_shape, num_coeffs).
    """
    basis_set = SphericalHarmonics(ell_max=ell_max)
    
    # Create projector (CUDA or CPU)
    if use_cuda:
        projector = SAXSProjectorCUDA(dc.geometry)
    else:
        projector = SAXSProjector(dc.geometry)
    
    residual_calculator = GradientResidualCalculator(
        data_container=dc,
        basis_set=basis_set,
        projector=projector
    )
    
    loss_function = SquaredLoss(residual_calculator)
    regularizer = Laplacian()
    loss_function.add_regularizer(
        name='laplacian',
        regularizer=regularizer,
        regularization_weight=regularization_weight
    )
    
    optimizer = LBFGS(loss_function, maxiter=maxiter)
    
    # Run optimization
    if verbose:
        print("Running LBFGS optimization...")
    results = optimizer.optimize()
    
    # Extract reconstruction
    if isinstance(results, dict):
        if 'x' in results:
            reconstruction = results['x']
        elif 'reconstruction' in results:
            reconstruction = results['reconstruction']
        elif 'coefficients' in results:
            reconstruction = results['coefficients']
        else:
            for key, value in results.items():
                if isinstance(value, (np.ndarray, torch.Tensor)) or hasattr(value, 'shape'):
                    reconstruction = value
                    if verbose:
                        print(f"Using key '{key}' as reconstruction")
                    break
            else:
                raise ValueError(f"Could not find reconstruction in results. Keys: {list(results.keys())}")
    elif hasattr(results, 'reconstruction'):
        reconstruction = results.reconstruction
    else:
        reconstruction = results
    
    # Convert to numpy if it's a torch tensor
    if torch.is_tensor(reconstruction):
        reconstruction = reconstruction.cpu().numpy()
    
    # Ensure it's a numpy array
    if not isinstance(reconstruction, np.ndarray):
        reconstruction = np.asarray(reconstruction)
    
    return reconstruction.astype(np.float32)


def _perform_reconstruction_pytorch(
    dc: DataContainer,
    ell_max: int,
    maxiter: int = 500,
    lr: float = 0.01,
    regularization_weight: float = 0.0,
    regularization_type: Literal['none', 'l2', 'tv', 'laplacian'] = 'none',
    use_cuda: bool = True,
    verbose: bool = False,
    log_interval: int = 50,
) -> np.ndarray:
    """
    Perform spherical harmonic reconstruction using PyTorch AdamW optimization.
    
    This function uses the PyTorch-accelerated forward projector and spherical
    harmonics forward operator to optimize the volume coefficients.
    
    Parameters
    ----------
    dc : DataContainer
        The data container with projections and geometry.
    ell_max : int
        Maximum degree for spherical harmonics expansion.
    maxiter : int, default=500
        Maximum number of AdamW iterations.
    lr : float, default=0.01
        Learning rate for AdamW optimizer.
    regularization_weight : float, default=0.0
        Weight for regularization term.
    regularization_type : str, default='none'
        Type of regularization: 'none', 'l2', 'tv' (total variation), 'laplacian'.
    use_cuda : bool, default=True
        Whether to use CUDA for computation.
    verbose : bool, default=False
        Whether to print progress information.
    log_interval : int, default=50
        How often to print loss during optimization.
    
    Returns
    -------
    np.ndarray
        The reconstruction as a numpy array of shape (*volume_shape, num_coeffs).
    """
    device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')
    
    # Get geometry info
    geometry = dc.geometry
    volume_shape = geometry.volume_shape
    num_coeffs = (ell_max // 2 + 1) * (ell_max + 1)
    
    # Get target data (measured projections after SH forward)
    # The data in dc.projections is already in detector segment space
    # Shape: (N, J, K, M) where N=projections, J,K=spatial, M=detector segments
    # Note: mumott ProjectionStack needs to be accessed element by element
    projection_data = np.stack([p.data for p in dc.projections])
    target_data = torch.tensor(
        projection_data.astype(np.float32), 
        device=device, 
        dtype=torch.float32
    )
    
    if verbose:
        print(f"Target data shape: {target_data.shape}")
        print(f"Volume shape: {volume_shape}")
        print(f"Number of SH coefficients: {num_coeffs}")
        print(f"Device: {device}")
    
    # Create basis set for SH forward (get projection matrix from mumott for exact matching)
    basis_set = SphericalHarmonics(ell_max=ell_max, probed_coordinates=geometry.probed_coordinates)
    projection_matrix = torch.tensor(
        basis_set.projection_matrix.astype(np.float32),
        device=device,
        dtype=torch.float32
    )
    
    if verbose:
        print(f"SH projection matrix shape: {projection_matrix.shape}")
    
    # Build differentiable projector (uses autograd with adjoint for backward)
    projector = build_mumott_projector(geometry, device=device)
    
    if verbose:
        print(f"Built differentiable projector")
    
    # Initialize volume coefficients (learnable parameters)
    # Shape: (*volume_shape, num_coeffs)
    volume = torch.zeros(
        (*volume_shape, num_coeffs),
        device=device,
        dtype=torch.float32,
        requires_grad=True
    )
    
    # Create optimizer
    optimizer = torch.optim.AdamW([volume], lr=lr, weight_decay=0.0)
    
    # Optional: learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=50
    )
    
    # Training loop
    losses = []
    
    if verbose:
        print(f"\nStarting PyTorch optimization ({maxiter} iterations)...")
        print(f"Regularization: {regularization_type} (weight={regularization_weight})")
    
    for iteration in range(maxiter):
        optimizer.zero_grad()
        
        # Forward projection: volume -> spatial projections
        # projector expects (X, Y, Z, C) and returns (N, J, K, C)
        # This uses build_mumott_projector which wraps ASTRA with autograd
        spatial_projections = projector(volume)
        
        # SH forward: spatial projections -> detector segments
        # forward_quadrature expects (N, H, W, C) and returns (N, H, W, M)
        # Keep projection_matrix on GPU for the einsum operation
        predicted = forward_quadrature(
            geometry.probed_coordinates,
            spatial_projections,
            ell_max=ell_max,
            projection_matrix=projection_matrix  # Use mumott's projection matrix (torch tensor)
        )
        
        # Data fidelity loss (MSE)
        data_loss = torch.nn.functional.mse_loss(predicted, target_data)
        
        # Regularization
        reg_loss = torch.tensor(0.0, device=device)
        
        if regularization_type == 'l2' and regularization_weight > 0:
            # L2 regularization on coefficients
            reg_loss = regularization_weight * torch.mean(volume ** 2)
            
        elif regularization_type == 'tv' and regularization_weight > 0:
            # Total variation regularization (anisotropic)
            dx = torch.diff(volume, dim=0)
            dy = torch.diff(volume, dim=1)
            dz = torch.diff(volume, dim=2)
            reg_loss = regularization_weight * (
                torch.mean(torch.abs(dx)) + 
                torch.mean(torch.abs(dy)) + 
                torch.mean(torch.abs(dz))
            )
            
        elif regularization_type == 'laplacian' and regularization_weight > 0:
            # Laplacian regularization (smoothness)
            # Approximate Laplacian using finite differences
            laplacian = torch.zeros_like(volume)
            # Interior points
            laplacian[1:-1, :, :, :] += volume[:-2, :, :, :] + volume[2:, :, :, :] - 2 * volume[1:-1, :, :, :]
            laplacian[:, 1:-1, :, :] += volume[:, :-2, :, :] + volume[:, 2:, :, :] - 2 * volume[:, 1:-1, :, :]
            laplacian[:, :, 1:-1, :] += volume[:, :, :-2, :] + volume[:, :, 2:, :] - 2 * volume[:, :, 1:-1, :]
            reg_loss = regularization_weight * torch.mean(laplacian ** 2)
        
        # Total loss
        total_loss = data_loss + reg_loss
        
        # Backward pass
        total_loss.backward()
        
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_([volume], max_norm=1.0)
        
        # Optimizer step
        optimizer.step()
        
        # Learning rate scheduler
        scheduler.step(total_loss.detach())
        
        # Logging
        losses.append(total_loss.item())
        
        if verbose and (iteration + 1) % log_interval == 0:
            print(f"  Iter {iteration + 1:4d}: loss={total_loss.item():.6f} "
                  f"(data={data_loss.item():.6f}, reg={reg_loss.item():.6f})")
    
    if verbose:
        print(f"\nOptimization complete. Final loss: {losses[-1]:.6f}")
    
    # Convert to numpy
    reconstruction = volume.detach().cpu().numpy()
    
    return reconstruction.astype(np.float32)


# Keep the original function name for backwards compatibility
def _perform_reconstruction(
    dc: DataContainer,
    ell_max: int,
    maxiter: int,
    regularization_weight: float,
    use_cuda: bool,
    verbose: bool = False,
    method: Literal['lbfgs', 'pytorch'] = 'lbfgs',
    **kwargs
) -> np.ndarray:
    """
    Perform a single spherical harmonic reconstruction.
    
    Parameters
    ----------
    dc : DataContainer
        The data container with projections.
    ell_max : int
        Maximum degree for spherical harmonics expansion.
    maxiter : int
        Maximum number of iterations.
    regularization_weight : float
        Weight for the regularization term.
    use_cuda : bool
        Whether to use CUDA for computation.
    verbose : bool, default=False
        Whether to print progress information.
    method : str, default='lbfgs'
        Optimization method: 'lbfgs' (mumott) or 'pytorch' (AdamW).
    **kwargs
        Additional arguments for the PyTorch method (lr, regularization_type, etc.)
    
    Returns
    -------
    np.ndarray
        The reconstruction as a numpy array of shape (*volume_shape, num_coeffs).
    """
    if method == 'lbfgs':
        return _perform_reconstruction_lbfgs(
            dc=dc,
            ell_max=ell_max,
            maxiter=maxiter,
            regularization_weight=regularization_weight,
            use_cuda=use_cuda,
            verbose=verbose
        )
    elif method == 'pytorch':
        return _perform_reconstruction_pytorch(
            dc=dc,
            ell_max=ell_max,
            maxiter=maxiter,
            regularization_weight=regularization_weight,
            use_cuda=use_cuda,
            verbose=verbose,
            **kwargs
        )
    else:
        raise ValueError(f"Unknown method: {method}. Use 'lbfgs' or 'pytorch'.")

## 1. Configuration

In [4]:
# Configuration - Using synthetic data for testing
# Since the original data file is not available, we'll generate synthetic data

# Reconstruction parameters
ell_max = 8
maxiter = 20
regularization_weight = 0 # usually 1.0 
use_cuda = True  # Set to False if CUDA is not available

# Test parameters
num_test_projections = 30  # Number of synthetic projections
test_volume_shape = (8, 8, 8)  # Small volume for quick testing

# Random seed for reproducibility
seed = 42
np.random.seed(seed)

print(f"Configuration (Synthetic Data Test):")
print(f"  ell_max: {ell_max}")
print(f"  Test volume shape: {test_volume_shape}")
print(f"  Test projections: {num_test_projections}")
print(f"  Using {'CUDA' if use_cuda else 'CPU'}")

Configuration (Synthetic Data Test):
  ell_max: 8
  Test volume shape: (8, 8, 8)
  Test projections: 30
  Using CUDA


In [5]:
# Load data from the frogbone dataset
data_path = '/myhome/data/smartt/shared/frogbone/dataset_qbin_0009.h5'

print(f"Loading data from {data_path}...")
print("=" * 60)

dc_full = DataContainer(data_path, nonfinite_replacement_value=0)
total_projections = len(dc_full.projections)

print(f"\nDataset information:")
print(f"  Total projections: {total_projections}")
print(f"  Volume shape: {dc_full.geometry.volume_shape}")
print(f"  Projection shape: {dc_full.geometry.projection_shape}")

# Create a subset for faster testing - use fewer projections to avoid memory issues
test_num_projections = 10
dc_test = copy.deepcopy(dc_full)

# Delete the full container to free memory
del dc_full

# Delete projections to keep only first test_num_projections
indices_to_delete = list(range(test_num_projections, total_projections))
for i in sorted(indices_to_delete, reverse=True):
    del dc_test.projections[i]

print(f"\nTest dataset:")
print(f"  Projections: {len(dc_test.projections)}")
print(f"  Volume shape: {dc_test.geometry.volume_shape}")

Loading data from /myhome/data/smartt/shared/frogbone/dataset_qbin_0009.h5...
INFO:Rotation matrices were loaded from the input file.


/opt/conda/lib/python3.10/site-packages/mumott/data_handling/data_container.py:227: DeprecationWarning: Entry name rotations is deprecated. Use inner_angle instead.
  _deprecated_key_warning('rotations')
/opt/conda/lib/python3.10/site-packages/mumott/data_handling/data_container.py:236: DeprecationWarning: Entry name tilts is deprecated. Use outer_angle instead.
  _deprecated_key_warning('tilts')
/opt/conda/lib/python3.10/site-packages/mumott/data_handling/data_container.py:246: DeprecationWarning: Entry name rot_mat is deprecated. Use rotation_matrix instead.
  _deprecated_key_warning('rot_mat')
/opt/conda/lib/python3.10/site-packages/mumott/data_handling/data_container.py:268: DeprecationWarning: Entry name offset_j is deprecated. Use j_offset instead.
  _deprecated_key_warning('offset_j')
/opt/conda/lib/python3.10/site-packages/mumott/data_handling/data_container.py:278: DeprecationWarning: Entry name offset_k is deprecated. Use k_offset instead.
  _deprecated_key_warning('offset_k'

INFO:Sample geometry loaded from file.
INFO:Detector geometry loaded from file.
INFO:Detector geometry loaded from file.

Dataset information:
  Total projections: 240
  Volume shape: [65 82 65]
  Projection shape: [ 73 100]

Test dataset:
  Projections: 10
  Volume shape: [65 82 65]

Dataset information:
  Total projections: 240
  Volume shape: [65 82 65]
  Projection shape: [ 73 100]

Test dataset:
  Projections: 10
  Volume shape: [65 82 65]


## 2. Load Dataset and Compute Ground Truth

We'll compute the ground truth reconstruction using all 240 available projections.

In [5]:
# Load data container
print("Loading data container...")
dc_full = DataContainer(data_path, nonfinite_replacement_value=0)
total_projections = len(dc_full.projections)

print(f"\nDataset information:")
print(f"  Total projections available: {total_projections}")
print(f"  Volume shape: {dc_full.geometry.volume_shape}")

Loading data container...


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/das/home/barbaf_l/p20639/Mads/frog/frogbone/dataset_qbin_0000.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## 2.1 Test PyTorch Reconstruction

Let's test the new PyTorch-accelerated reconstruction and compare it with the LBFGS method.

In [6]:
# Test PyTorch reconstruction on the loaded test dataset
print("Testing PyTorch reconstruction...")
print("=" * 60)

print(f"Test dataset: {len(dc_test.projections)} projections")
print(f"Volume shape: {dc_test.geometry.volume_shape}")

# Test PyTorch reconstruction
print("\n--- Testing PyTorch AdamW Reconstruction ---")
recon_pytorch = _perform_reconstruction_pytorch(
    dc=dc_test,
    ell_max=ell_max,
    maxiter=100,  # Fewer iterations for test
    lr=0.01,
    regularization_weight=0.0,
    regularization_type='none',
    use_cuda=True,
    verbose=True,
    log_interval=10
)
print(f"\nPyTorch reconstruction shape: {recon_pytorch.shape}")
print(f"Reconstruction stats: min={recon_pytorch.min():.4f}, max={recon_pytorch.max():.4f}, mean={recon_pytorch.mean():.4f}")

Testing PyTorch reconstruction...
Test dataset: 10 projections
Volume shape: [65 82 65]

--- Testing PyTorch AdamW Reconstruction ---
Target data shape: torch.Size([10, 73, 100, 8])
Volume shape: [65 82 65]
Number of SH coefficients: 45
Device: cuda


/opt/conda/lib/python3.10/site-packages/mumott/methods/basis_sets/spherical_harmonics.py:103: DeprecationWarning: `scipy.special.sph_harm` is deprecated as of SciPy 1.15.0 and will be removed in SciPy 1.17.0. Please use `scipy.special.sph_harm_y` instead.
  complex_factors = sph_harm(abs(self._emm_indices)[np.newaxis, np.newaxis, np.newaxis, ...],


SH projection matrix shape: torch.Size([10, 8, 45])
Built differentiable projector

Starting PyTorch optimization (100 iterations)...
Regularization: none (weight=0.0)
  Iter   10: loss=317045.468750 (data=317045.468750, reg=0.000000)
  Iter   10: loss=317045.468750 (data=317045.468750, reg=0.000000)
  Iter   20: loss=288696.218750 (data=288696.218750, reg=0.000000)
  Iter   20: loss=288696.218750 (data=288696.218750, reg=0.000000)
  Iter   30: loss=263326.500000 (data=263326.500000, reg=0.000000)
  Iter   30: loss=263326.500000 (data=263326.500000, reg=0.000000)
  Iter   40: loss=240478.015625 (data=240478.015625, reg=0.000000)
  Iter   40: loss=240478.015625 (data=240478.015625, reg=0.000000)
  Iter   50: loss=219952.421875 (data=219952.421875, reg=0.000000)
  Iter   50: loss=219952.421875 (data=219952.421875, reg=0.000000)
  Iter   60: loss=201604.109375 (data=201604.109375, reg=0.000000)
  Iter   60: loss=201604.109375 (data=201604.109375, reg=0.000000)
  Iter   70: loss=185313.187

In [7]:
# Performance Summary after ASTRA geometry caching optimization
print("=" * 60)
print("Performance Optimization Summary")
print("=" * 60)

# The optimization was caching ASTRA geometries in _AstraMumottOp
# Before: geometry created for each of 45 coefficients per forward/backward
# After: geometry cached once, reused for all 45 coefficients

print("""
Key Changes Made:
-----------------
1. Added cached ASTRA geometries in _AstraMumottOp.__init__()
2. Added forward_single() and forward_batched() methods using cached geometry
3. Added adjoint_single() and adjoint_batched() methods using cached geometry  
4. Added helper functions _backproject_single_gpu/cpu with geometry parameters

Performance Results:
--------------------
Before optimization:
  - Forward projection (45 coeffs): ~590ms per iteration
  - Backward pass: ~625ms per iteration (estimated)
  - Full iteration: ~1.2s

After optimization:  
  - Forward projection (45 coeffs): ~185-189ms per iteration (3.1x faster)
  - Backward pass: ~416ms per iteration (1.5x faster)
  - Full iteration: ~410ms (2.9x faster overall)

Single ASTRA call timing: ~3.3ms
  - Minimum theoretical time for 45 calls: 149ms
  - Actual forward time: 185-189ms (27% overhead - acceptable)

The remaining bottleneck is the serial nature of ASTRA:
  - Each coefficient requires its own ASTRA forward/backward call
  - 45 coefficients = 45 serial ASTRA calls
  - Cannot be parallelized within a single ASTRA instance
""")

Performance Optimization Summary

Key Changes Made:
-----------------
1. Added cached ASTRA geometries in _AstraMumottOp.__init__()
2. Added forward_single() and forward_batched() methods using cached geometry
3. Added adjoint_single() and adjoint_batched() methods using cached geometry  
4. Added helper functions _backproject_single_gpu/cpu with geometry parameters

Performance Results:
--------------------
Before optimization:
  - Forward projection (45 coeffs): ~590ms per iteration
  - Backward pass: ~625ms per iteration (estimated)
  - Full iteration: ~1.2s

After optimization:  
  - Forward projection (45 coeffs): ~185-189ms per iteration (3.1x faster)
  - Backward pass: ~416ms per iteration (1.5x faster)
  - Full iteration: ~410ms (2.9x faster overall)

Single ASTRA call timing: ~3.3ms
  - Minimum theoretical time for 45 calls: 149ms
  - Actual forward time: 185-189ms (27% overhead - acceptable)

The remaining bottleneck is the serial nature of ASTRA:
  - Each coefficient require

In [14]:
# Performance profiling of forward and backward projections
import time

print("=" * 60)
print("Performance Profiling")
print("=" * 60)

# Clear GPU memory
torch.cuda.empty_cache()
torch.cuda.synchronize()

# Setup
device = torch.device('cuda')
geometry = dc_test.geometry
volume_shape = tuple(geometry.volume_shape)
num_coeffs = 45

# Create test volume
test_volume = torch.randn(*volume_shape, num_coeffs, device=device, dtype=torch.float32, requires_grad=True)

# Build projector
projector = build_mumott_projector(geometry, device=device)

# Get projection matrix for SH forward
basis_set = SphericalHarmonics(ell_max=8, probed_coordinates=geometry.probed_coordinates)
projection_matrix = torch.tensor(basis_set.projection_matrix.astype(np.float32), device=device)

print(f"\nTest setup:")
print(f"  Volume shape: {test_volume.shape}")
print(f"  Projection matrix shape: {projection_matrix.shape}")
print(f"  Device: {device}")

# Warm up
print("\n--- Warm-up run ---")
_ = projector(test_volume)
torch.cuda.synchronize()

# Test 1: Forward projection only (ASTRA)
print("\n--- Test 1: ASTRA Forward Projection ---")
torch.cuda.synchronize()
start = time.time()
for i in range(5):
    spatial_proj = projector(test_volume)
    torch.cuda.synchronize()
end = time.time()
print(f"  5 forward projections: {end - start:.3f}s ({(end-start)/5*1000:.1f}ms per iteration)")
print(f"  Output shape: {spatial_proj.shape}")

# Test 2: SH Forward (einsum)
print("\n--- Test 2: SH Forward (einsum) ---")
torch.cuda.synchronize()
start = time.time()
for i in range(5):
    predicted = forward_quadrature(
        geometry.probed_coordinates,
        spatial_proj,
        ell_max=8,
        projection_matrix=projection_matrix
    )
    torch.cuda.synchronize()
end = time.time()
print(f"  5 SH forward ops: {end - start:.3f}s ({(end-start)/5*1000:.1f}ms per iteration)")
print(f"  Output shape: {predicted.shape}")

# Test 3: Full forward pass
print("\n--- Test 3: Full Forward Pass ---")
torch.cuda.synchronize()
start = time.time()
for i in range(5):
    spatial_proj = projector(test_volume)
    predicted = forward_quadrature(
        geometry.probed_coordinates,
        spatial_proj,
        ell_max=8,
        projection_matrix=projection_matrix
    )
    torch.cuda.synchronize()
end = time.time()
print(f"  5 full forward passes: {end - start:.3f}s ({(end-start)/5*1000:.1f}ms per iteration)")

# Test 4: Backward pass
print("\n--- Test 4: Backward Pass ---")
target = torch.randn_like(predicted)
torch.cuda.synchronize()
start = time.time()
for i in range(5):
    test_volume.grad = None
    spatial_proj = projector(test_volume)
    predicted = forward_quadrature(
        geometry.probed_coordinates,
        spatial_proj,
        ell_max=8,
        projection_matrix=projection_matrix
    )
    loss = torch.nn.functional.mse_loss(predicted, target)
    loss.backward()
    torch.cuda.synchronize()
end = time.time()
print(f"  5 forward+backward: {end - start:.3f}s ({(end-start)/5*1000:.1f}ms per iteration)")

print("\n" + "=" * 60)

Performance Profiling

Test setup:
  Volume shape: torch.Size([65, 82, 65, 45])
  Projection matrix shape: torch.Size([10, 8, 45])
  Device: cuda

--- Warm-up run ---

--- Test 1: ASTRA Forward Projection ---

--- Test 1: ASTRA Forward Projection ---
  5 forward projections: 0.926s (185.2ms per iteration)
  Output shape: torch.Size([10, 73, 100, 45])

--- Test 2: SH Forward (einsum) ---
  5 SH forward ops: 0.004s (0.7ms per iteration)
  Output shape: torch.Size([10, 73, 100, 8])

--- Test 3: Full Forward Pass ---
  5 forward projections: 0.926s (185.2ms per iteration)
  Output shape: torch.Size([10, 73, 100, 45])

--- Test 2: SH Forward (einsum) ---
  5 SH forward ops: 0.004s (0.7ms per iteration)
  Output shape: torch.Size([10, 73, 100, 8])

--- Test 3: Full Forward Pass ---
  5 full forward passes: 0.962s (192.3ms per iteration)

--- Test 4: Backward Pass ---
  5 full forward passes: 0.962s (192.3ms per iteration)

--- Test 4: Backward Pass ---
  5 forward+backward: 2.080s (415.9ms p

In [10]:
# Check what's happening with batching
print("Investigating the batching issue...")
print(f"test_volume.ndim = {test_volume.ndim}")
print(f"test_volume.shape = {test_volume.shape}")
print(f"Last dimension (coefficients) = {test_volume.shape[-1]}")
print("\nThe projector sees this as 45 BATCHES, calling ASTRA 45 times!")
print("This is the bottleneck - we need to vectorize over coefficients.")

Investigating the batching issue...
test_volume.ndim = 4
test_volume.shape = torch.Size([65, 82, 65, 45])
Last dimension (coefficients) = 45

The projector sees this as 45 BATCHES, calling ASTRA 45 times!
This is the bottleneck - we need to vectorize over coefficients.


In [13]:
# Reload the projector module after changes
import importlib
import smartt.projectors.astra_projector
importlib.reload(smartt.projectors.astra_projector)
from smartt.projectors.astra_projector import forward_project, build_mumott_projector

print("Module reloaded successfully")

Module reloaded successfully


In [15]:
# Compare single vs batched projection performance
import time

print("=" * 60)
print("Single vs Batched Projection Timing")
print("=" * 60)

# Single 3D volume (no batch dimension)
test_single = torch.randn(*volume_shape, device=device, dtype=torch.float32, requires_grad=True)

# Build projector (this caches geometries)
projector = build_mumott_projector(geometry, device=device)

# Warm up
_ = projector(test_single)
torch.cuda.synchronize()

# Test single volume (1 ASTRA call)
print("\n--- Single Volume (1 ASTRA call) ---")
torch.cuda.synchronize()
start = time.time()
for i in range(10):
    _ = projector(test_single)
    torch.cuda.synchronize()
end = time.time()
single_time = (end - start) / 10
print(f"  10 single projections: {end - start:.3f}s ({single_time*1000:.1f}ms per projection)")

# Test batched volume (45 ASTRA calls)
test_batched = torch.randn(*volume_shape, 45, device=device, dtype=torch.float32, requires_grad=True)
print(f"\n--- Batched Volume (45 ASTRA calls, shape {test_batched.shape}) ---")
torch.cuda.synchronize()
start = time.time()
for i in range(5):
    _ = projector(test_batched)
    torch.cuda.synchronize()
end = time.time()
batched_time = (end - start) / 5
print(f"  5 batched projections: {end - start:.3f}s ({batched_time*1000:.1f}ms per batch)")

print(f"\n--- Analysis ---")
print(f"  Time per single ASTRA call: {single_time*1000:.1f}ms")
print(f"  Time per batched call (45 ASTRA): {batched_time*1000:.1f}ms")
print(f"  Overhead per batch: {batched_time*1000 - 45*single_time*1000:.1f}ms (should be ~0)")
print(f"  Expected time if 45 calls: {45*single_time*1000:.1f}ms")
print(f"  Actual overhead factor: {batched_time / (45*single_time):.2f}x")

Single vs Batched Projection Timing

--- Single Volume (1 ASTRA call) ---
  10 single projections: 0.033s (3.3ms per projection)

--- Batched Volume (45 ASTRA calls, shape torch.Size([65, 82, 65, 45])) ---
  5 batched projections: 0.944s (188.9ms per batch)

--- Analysis ---
  Time per single ASTRA call: 3.3ms
  Time per batched call (45 ASTRA): 188.9ms
  Overhead per batch: 39.7ms (should be ~0)
  Expected time if 45 calls: 149.1ms
  Actual overhead factor: 1.27x


In [17]:
# Let's also check what mumott's native SAXSProjectorCUDA does
from mumott.methods.projectors import SAXSProjectorCUDA

print("=" * 60)
print("mumott SAXSProjectorCUDA Comparison")
print("=" * 60)

# Create SAXSProjectorCUDA
saxs_proj = SAXSProjectorCUDA(geometry)

# Create volume in mumott format 
# mumott expects: (nx, ny, nz, n_coeffs)
vol_mumott = np.random.randn(*volume_shape, 45).astype(np.float32)
print(f"mumott volume shape: {vol_mumott.shape}")
print(f"geometry.volume_shape: {geometry.volume_shape}")

# Warm up
_ = saxs_proj.forward(vol_mumott)

# Time mumott's forward
print("\n--- mumott SAXSProjectorCUDA.forward ---")
start = time.time()
for i in range(5):
    result = saxs_proj.forward(vol_mumott)
end = time.time()
print(f"  5 forward projections: {end - start:.3f}s ({(end-start)/5*1000:.1f}ms per iteration)")
print(f"  Output shape: {result.shape}")

# Compare to our implementation
print("\n--- Our build_mumott_projector ---")
torch.cuda.synchronize()
start = time.time()
for i in range(5):
    _ = projector(test_batched)
    torch.cuda.synchronize()
end = time.time()
print(f"  5 forward projections: {end - start:.3f}s ({(end-start)/5*1000:.1f}ms per iteration)")

mumott SAXSProjectorCUDA Comparison


: 

In [ ]:
# Compute ground truth reconstruction with all projections
print(f"\nComputing ground truth reconstruction with all {total_projections} projections...")
print("This may take several minutes...\n")

ground_truth = _perform_reconstruction(
    dc=dc_full,
    ell_max=ell_max,
    maxiter=maxiter,
    regularization_weight=regularization_weight,
    use_cuda=use_cuda,
    verbose=True
)

print(f"\nGround truth shape: {ground_truth.shape}")
volume_shape = ground_truth.shape[:3]
num_coeffs = ground_truth.shape[3]
print(f"Volume shape: {volume_shape}")
print(f"Number of coefficients: {num_coeffs}")

## 3. Select Fixed Sparse Subset

Randomly select 60 projections from the full set. This subset will remain fixed for all experiments.

In [ ]:
# Randomly select fixed sparse subset of 60 projections
np.random.seed(seed)
fixed_sparse_indices = np.random.choice(
    total_projections,
    size=num_projections_sparse,
    replace=False
)
fixed_sparse_indices = np.arange(num_projections_sparse)

fixed_sparse_indices = np.sort(fixed_sparse_indices)

fixed_sparse_indices = np.linspace(0, 50, num_projections_sparse, dtype=int)

print(f"Fixed sparse subset (60 projections):")
print(f"  Indices: {fixed_sparse_indices[:10]}...{fixed_sparse_indices[-10:]}")
print(f"  Min: {fixed_sparse_indices.min()}, Max: {fixed_sparse_indices.max()}")

## 4. Run Multiple Sparse Reconstruction Experiments

For each experiment:
1. Randomly sample 80% of the fixed 60 projections (48 projections)
2. Perform sparse reconstruction with these 48 projections
3. Store the result

In [ ]:
# Pre-allocate array to store all reconstructions
all_reconstructions = np.zeros(
    (num_experiments, *volume_shape, num_coeffs),
    dtype=np.float32
)

# Store the indices used in each experiment
subsample_indices_list = []

print(f"\nRunning {num_experiments} sparse reconstruction experiments...")
print(f"Each experiment uses {num_subsamples} randomly sampled projections from the fixed subset of 60.\n")

for exp_idx in tqdm(range(num_experiments), desc="Experiments"):
    # Randomly sample 80% of the fixed sparse subset
    subsample_indices = np.random.choice(
        fixed_sparse_indices,
        size=num_subsamples,
        replace=False
    )

    
    subsample_indices = np.sort(subsample_indices)
    subsample_indices_list.append(subsample_indices)
    
    # Create a fresh data container with only the subsampled projections
    dc_subsample = DataContainer(data_path, nonfinite_replacement_value=0)
    
    # Remove projections not in the subsample
    all_indices = np.arange(total_projections)
    indices_to_delete = [i for i in all_indices if i not in subsample_indices]
    
    for i in sorted(indices_to_delete, reverse=True):
        del dc_subsample.projections[i]
    
    # Perform reconstruction
    reconstruction = _perform_reconstruction(
        dc=dc_subsample,
        ell_max=ell_max,
        maxiter=maxiter,
        regularization_weight=regularization_weight,
        use_cuda=use_cuda,
        verbose=False
    )
    
    # Store reconstruction
    all_reconstructions[exp_idx] = reconstruction
    
    # Clean up
    del dc_subsample, reconstruction

print(f"\nCompleted {num_experiments} experiments.")
print(f"All reconstructions shape: {all_reconstructions.shape}")

In [ ]:
# Plot first 10 slices of all_reconstructions
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

for i in range(10):
    if i < num_experiments:
        slice_data = all_reconstructions[i, 32, :, :, 0]
        im = axes[i].imshow(slice_data, cmap='viridis')
        axes[i].set_title(f'Experiment {i}', fontsize=11, fontweight='bold')
        axes[i].axis('off')
        plt.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)
    else:
        axes[i].axis('off')

fig.suptitle('First 10 Reconstructions at z=32, coefficient 0', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Compute Standard Deviation Across Experiments

Calculate the standard deviation across all experiments to quantify reconstruction uncertainty.

In [ ]:
# Compute mean and standard deviation across experiments
mean_reconstruction = np.mean(all_reconstructions, axis=0)
std_reconstruction = np.std(all_reconstructions, axis=0)

print(f"Mean reconstruction shape: {mean_reconstruction.shape}")
print(f"Std reconstruction shape: {std_reconstruction.shape}")

# Compute statistics
print(f"\nStandard deviation statistics (across all voxels and coefficients):")
print(f"  Mean std: {np.mean(std_reconstruction):.6f}")
print(f"  Median std: {np.median(std_reconstruction):.6f}")
print(f"  Min std: {np.min(std_reconstruction):.6f}")
print(f"  Max std: {np.max(std_reconstruction):.6f}")

# Per-coefficient statistics
print(f"\nPer-coefficient std statistics:")
for coeff_idx in range(num_coeffs):
    coeff_std = std_reconstruction[..., coeff_idx]
    print(f"  Coefficient {coeff_idx:2d}: mean={np.mean(coeff_std):.6f}, "
          f"median={np.median(coeff_std):.6f}, max={np.max(coeff_std):.6f}")

## 6. Visualize Standard Deviation Slices

Plot slices through the 3D standard deviation volume for different spherical harmonic coefficients.

In [ ]:
# Select coefficients to visualize
# Coefficient 0 is l=0 (isotropic), then l=2, l=4, l=6, l=8
coeffs_to_plot = [0, 1, 6, 15, 28, 44]  # Start of each l value: l=0, l=2, l=4, l=6, l=8, last coeff
coeff_labels = ['ℓ=0 (iso)', 'ℓ=2, m=-2', 'ℓ=4, m=-4', 'ℓ=6, m=-6', 'ℓ=8, m=-8', 'ℓ=8, m=8']

# Select middle slice
mid_z = volume_shape[2] // 2

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, (coeff_idx, label) in enumerate(zip(coeffs_to_plot, coeff_labels)):
    std_slice = std_reconstruction[:, :, mid_z, coeff_idx]
    
    im = axes[idx].imshow(std_slice.T, cmap='hot', origin='lower')
    axes[idx].set_title(f'Std Dev - {label}\n(z={mid_z})', fontsize=12, fontweight='bold')
    axes[idx].axis('off')
    plt.colorbar(im, ax=axes[idx], fraction=0.046, pad=0.04)

fig.suptitle(f'Standard Deviation Across {num_experiments} Reconstructions\n'
             f'(Each with {num_subsamples} random projections from fixed subset of {num_projections_sparse})',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Multiple Z-Slices for Selected Coefficients

In [ ]:
# Plot multiple z-slices for the isotropic component (l=0)
coeff_idx = 1
z_slices = [volume_shape[2]//4, volume_shape[2]//2, 3*volume_shape[2]//4]

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, z in enumerate(z_slices):
    std_slice = std_reconstruction[:, :, z, coeff_idx]
    
    im = axes[idx].imshow(std_slice.T, cmap='hot', origin='lower')
    axes[idx].set_title(f'Std Dev - ℓ=0 (isotropic)\nz={z}', fontsize=12, fontweight='bold')
    axes[idx].axis('off')
    plt.colorbar(im, ax=axes[idx], fraction=0.046, pad=0.04)

fig.suptitle(f'Standard Deviation Across Different Z-Slices', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Compare Mean Reconstruction vs Ground Truth

In [ ]:
# Compare mean of sparse reconstructions to ground truth
coeff_idx = 2  # Isotropic component
mid_z = volume_shape[2] // 2

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Determine shared colorbar limits
vmin = min(mean_reconstruction[:, :, mid_z, coeff_idx].min(),
           ground_truth[:, :, mid_z, coeff_idx].min())
vmax = max(mean_reconstruction[:, :, mid_z, coeff_idx].max(),
           ground_truth[:, :, mid_z, coeff_idx].max())

# Mean reconstruction
im0 = axes[0].imshow(mean_reconstruction[:, :, mid_z, coeff_idx].T,
                     cmap='viridis', vmin=vmin, vmax=vmax, origin='lower')
axes[0].set_title(f'Mean of {num_experiments} Sparse\nReconstructions (ℓ=0)', fontsize=12, fontweight='bold')
axes[0].axis('off')
plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

# Ground truth
im1 = axes[1].imshow(ground_truth[:, :, mid_z, coeff_idx].T,
                     cmap='viridis', vmin=vmin, vmax=vmax, origin='lower')
axes[1].set_title(f'Ground Truth\n(All {total_projections} projections, ℓ=0)', fontsize=12, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

# Difference
diff = mean_reconstruction[:, :, mid_z, coeff_idx] - ground_truth[:, :, mid_z, coeff_idx]
im2 = axes[2].imshow(diff.T, cmap='RdBu_r', origin='lower',
                     vmin=-np.abs(diff).max(), vmax=np.abs(diff).max())
axes[2].set_title('Difference\n(Mean - Ground Truth)', fontsize=12, fontweight='bold')
axes[2].axis('off')
plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)

fig.suptitle(f'Mean Sparse Reconstruction vs Ground Truth (z={mid_z})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Compute error metrics
mse = np.mean((mean_reconstruction - ground_truth)**2)
mae = np.mean(np.abs(mean_reconstruction - ground_truth))
print(f"\nError metrics (Mean reconstruction vs Ground truth):")
print(f"  MSE: {mse:.6f}")
print(f"  MAE: {mae:.6f}")

## 9. Histogram of Standard Deviations

In [ ]:
# Plot histograms of standard deviations for different coefficients
coeffs_to_plot = [0, 1, 6, 15, 28, 44]
coeff_labels = ['ℓ=0', 'ℓ=2, m=-2', 'ℓ=4, m=-4', 'ℓ=6, m=-6', 'ℓ=8, m=-8', 'ℓ=8, m=8']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, (coeff_idx, label) in enumerate(zip(coeffs_to_plot, coeff_labels)):
    std_flat = std_reconstruction[..., coeff_idx].flatten()
    
    axes[idx].hist(std_flat, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[idx].set_xlabel('Standard Deviation', fontsize=10)
    axes[idx].set_ylabel('Frequency', fontsize=10)
    axes[idx].set_title(f'{label}', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    
    # Add statistics
    mean_std = np.mean(std_flat)
    median_std = np.median(std_flat)
    axes[idx].axvline(mean_std, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_std:.4f}')
    axes[idx].axvline(median_std, color='orange', linestyle='--', linewidth=2, label=f'Median: {median_std:.4f}')
    axes[idx].legend(fontsize=9)

fig.suptitle(f'Distribution of Standard Deviations Across Voxels', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Coefficient-wise Analysis

In [ ]:
# Compute mean std for each coefficient
mean_std_per_coeff = np.array([np.mean(std_reconstruction[..., i]) for i in range(num_coeffs)])
median_std_per_coeff = np.array([np.median(std_reconstruction[..., i]) for i in range(num_coeffs)])
max_std_per_coeff = np.array([np.max(std_reconstruction[..., i]) for i in range(num_coeffs)])

fig, axes = plt.subplots(3, 1, figsize=(14, 12))

x = np.arange(num_coeffs)

# Mean std per coefficient
axes[0].bar(x, mean_std_per_coeff, color='steelblue', alpha=0.7)
axes[0].set_xlabel('Coefficient Index', fontsize=11)
axes[0].set_ylabel('Mean Std Dev', fontsize=11)
axes[0].set_title('Mean Standard Deviation per Coefficient', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Median std per coefficient
axes[1].bar(x, median_std_per_coeff, color='orange', alpha=0.7)
axes[1].set_xlabel('Coefficient Index', fontsize=11)
axes[1].set_ylabel('Median Std Dev', fontsize=11)
axes[1].set_title('Median Standard Deviation per Coefficient', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Max std per coefficient
axes[2].bar(x, max_std_per_coeff, color='crimson', alpha=0.7)
axes[2].set_xlabel('Coefficient Index', fontsize=11)
axes[2].set_ylabel('Max Std Dev', fontsize=11)
axes[2].set_title('Maximum Standard Deviation per Coefficient', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3, axis='y')

# Add ℓ value labels
for ax in axes:
    ax.set_xticks([0, 1, 6, 15, 28])
    ax.set_xticklabels(['ℓ=0', 'ℓ=2', 'ℓ=4', 'ℓ=6', 'ℓ=8'])

plt.tight_layout()
plt.show()

## 11. Relative Uncertainty

Compute relative uncertainty (std / mean) to understand uncertainty relative to signal strength.

In [ ]:
# Compute relative uncertainty (coefficient of variation)
# Add small epsilon to avoid division by zero
epsilon = 1e-10
relative_uncertainty = std_reconstruction / (np.abs(mean_reconstruction) + epsilon)

# Clip extreme values for visualization
relative_uncertainty_clipped = np.clip(relative_uncertainty, 0, 5)

print(f"Relative uncertainty statistics:")
print(f"  Mean: {np.mean(relative_uncertainty):.4f}")
print(f"  Median: {np.median(relative_uncertainty):.4f}")
print(f"  90th percentile: {np.percentile(relative_uncertainty, 90):.4f}")
print(f"  99th percentile: {np.percentile(relative_uncertainty, 99):.4f}")

In [ ]:
# Visualize relative uncertainty for selected coefficients
coeffs_to_plot = [0, 1, 6, 15, 28, 44]
coeff_labels = ['ℓ=0', 'ℓ=2, m=-2', 'ℓ=4, m=-4', 'ℓ=6, m=-6', 'ℓ=8, m=-8', 'ℓ=8, m=8']
mid_z = volume_shape[2] // 2

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, (coeff_idx, label) in enumerate(zip(coeffs_to_plot, coeff_labels)):
    rel_unc_slice = relative_uncertainty_clipped[:, :, mid_z, coeff_idx]
    
    im = axes[idx].imshow(rel_unc_slice.T, cmap='plasma', origin='lower', vmin=0, vmax=2)
    axes[idx].set_title(f'Relative Uncertainty - {label}\n(z={mid_z})', fontsize=12, fontweight='bold')
    axes[idx].axis('off')
    plt.colorbar(im, ax=axes[idx], fraction=0.046, pad=0.04)

fig.suptitle(f'Relative Uncertainty (Std / |Mean|) - Clipped to [0, 2]', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 12. Save Results (Optional)

In [ ]:
# Optionally save results to HDF5
save_results = True  # Set to True to save
output_path = '/myhome/smartt/results/sparse_uncertainty_analysis.h5'

if save_results:
    print(f"Saving results to {output_path}...")
    
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    
    with h5py.File(output_path, 'w') as f:
        # Save reconstructions
        f.create_dataset('all_reconstructions', data=all_reconstructions, compression='gzip')
        f.create_dataset('mean_reconstruction', data=mean_reconstruction, compression='gzip')
        f.create_dataset('std_reconstruction', data=std_reconstruction, compression='gzip')
        f.create_dataset('ground_truth', data=ground_truth, compression='gzip')
        
        # Save indices
        f.create_dataset('fixed_sparse_indices', data=fixed_sparse_indices)
        for i, indices in enumerate(subsample_indices_list):
            f.create_dataset(f'subsample_indices_{i}', data=indices)
        
        # Save metadata
        f.attrs['ell_max'] = ell_max
        f.attrs['num_experiments'] = num_experiments
        f.attrs['num_projections_sparse'] = num_projections_sparse
        f.attrs['num_subsamples'] = num_subsamples
        f.attrs['total_projections'] = total_projections
        f.attrs['volume_shape'] = volume_shape
        f.attrs['num_coeffs'] = num_coeffs
        f.attrs['seed'] = seed
    
    print(f"Results saved successfully!")
else:
    print("Results not saved (set save_results=True to save)")

In [ ]:
# Load saved results from HDF5
load_results = False  # Set to True to load
input_path = '/myhome/smartt/results/sparse_uncertainty_analysis.h5'

if load_results:
    print(f"Loading results from {input_path}...")
    
    with h5py.File(input_path, 'r') as f:
        # Load reconstructions
        all_reconstructions = f['all_reconstructions'][:]
        mean_reconstruction = f['mean_reconstruction'][:]
        std_reconstruction = f['std_reconstruction'][:]
        ground_truth = f['ground_truth'][:]
        
        # Load indices
        fixed_sparse_indices = f['fixed_sparse_indices'][:]
        subsample_indices_list = []
        for i in range(f.attrs['num_experiments']):
            subsample_indices = f[f'subsample_indices_{i}'][:]
            subsample_indices_list.append(subsample_indices)
        
        # Load metadata
        ell_max = f.attrs['ell_max']
        num_experiments = f.attrs['num_experiments']
        num_projections_sparse = f.attrs['num_projections_sparse']
        num_subsamples = f.attrs['num_subsamples']
        total_projections = f.attrs['total_projections']
        volume_shape = tuple(f.attrs['volume_shape'])
        num_coeffs = f.attrs['num_coeffs']
        seed = f.attrs['seed']
    
    print(f"Results loaded successfully!")
    print(f"\nLoaded data:")
    print(f"  all_reconstructions shape: {all_reconstructions.shape}")
    print(f"  mean_reconstruction shape: {mean_reconstruction.shape}")
    print(f"  std_reconstruction shape: {std_reconstruction.shape}")
    print(f"  ground_truth shape: {ground_truth.shape}")
    print(f"  Number of experiments: {num_experiments}")
    print(f"  Fixed sparse subset: {len(fixed_sparse_indices)} projections")
    print(f"  Subsample size: {num_subsamples} projections")
else:
    print("Results not loaded (set load_results=True to load)")

## Summary

This notebook demonstrates:
1. **Ground Truth**: Computed using all 240 available projections
2. **Fixed Sparse Subset**: 60 projections randomly selected and fixed
3. **Multiple Experiments**: 20 reconstructions, each using 48 randomly sampled projections from the fixed subset
4. **Uncertainty Quantification**: Standard deviation across experiments shows reconstruction uncertainty
5. **Coefficient Analysis**: Different spherical harmonic coefficients show varying levels of uncertainty
6. **Relative Uncertainty**: Normalized by signal strength to identify problematic regions

Key findings:
- Higher-order coefficients (larger ℓ values) typically show higher uncertainty
- Regions with low signal show higher relative uncertainty
- The standard deviation provides a measure of reconstruction stability given sparse sampling

## 13. Forward Project Reconstructions to Analyze Angular Variability

We'll project each reconstruction through all projection angles and analyze which angles show the highest variability across experiments.

In [ ]:
subset_vecs = dc_full.geometry.probed_coordinates.vector[69:75]
points3D = torch.tensor(subset_vecs).reshape(-1, 3)
points3D


In [ ]:
import torch
import lovely_tensors as lt
lt.monkey_patch()
colors = torch.arange(len(subset_vecs)).repeat_interleave(24)
colors = colors.reshape(-1)
colors, torch.tensor(points3D)

In [ ]:
import torch
# Create 3D plot of probed coordinates
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Create sphere surface
u = np.linspace(0, 2 * np.pi, 50)
v = np.linspace(0, np.pi, 50)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))

# Plot translucent sphere
ax.plot_surface(x_sphere, y_sphere, z_sphere, color='cyan', alpha=0.1, edgecolor='none')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Translucent Sphere (Radius = 1)', fontsize=14, fontweight='bold')
ax.set_box_aspect([1,1,1])

# Plot the 3D points
ax.scatter(points3D[:, 0], points3D[:, 1], points3D[:, 2], 
           c=colors, s=10, cmap='viridis', alpha=0.6)

ax.set_xlabel('X', fontsize=11)
ax.set_ylabel('Y', fontsize=11)
ax.set_zlabel('Z', fontsize=11)
ax.set_title('Probed Coordinates in 3D Space', fontsize=14, fontweight='bold')
ax.view_init(elev=20, azim=45)

plt.tight_layout()
plt.show()

print(f"Total number of probed points: {len(points3D)}")
print(f"Points shape: {points3D.shape}")

In [ ]:
# Create projector using the full geometry from dc_full
print("Creating projector with full geometry...")
projector_full = SAXSProjector(dc_full.geometry)

# Create basis set for forward projection
basis_set = SphericalHarmonics(ell_max=ell_max, probed_coordinates=dc_full.geometry.probed_coordinates)

print(f"Projector geometry:")
print(f"  Number of projections: {len(dc_full.geometry.inner_angles)}")
print(f"  Projection shape: {dc_full.geometry.projection_shape}")
print(f"  Detector angles: {dc_full.geometry.detector_angles.shape}")

num_angles = len(dc_full.geometry.inner_angles)
proj_shape = dc_full.geometry.projection_shape
num_detector_angles = len(dc_full.geometry.detector_angles)

print(f"\nExpected output shape per reconstruction: ({num_angles}, {proj_shape[0]}, {proj_shape[1]}, {num_detector_angles})")
print(f"Expected output shape for all reconstructions: ({num_experiments}, {num_angles}, {proj_shape[0]}, {proj_shape[1]}, {num_detector_angles})")

In [ ]:
# Forward project all reconstructions
print(f"\nForward projecting {num_experiments} reconstructions through {num_angles} angles...")
print("This may take several minutes...\n")

# Pre-allocate array for all forward projections
all_forward_projections = np.zeros(
    (num_experiments, num_angles, proj_shape[0], proj_shape[1], num_detector_angles),
    dtype=np.float32
)

for exp_idx in tqdm(range(num_experiments), desc="Forward projecting"):
    reconstruction = all_reconstructions[exp_idx].astype(np.float64)
    
    # Forward project: projector.forward() then basis_set.forward()
    spatial_projection = projector_full.forward(reconstruction)
    forward_proj = basis_set.forward(spatial_projection)
    
    all_forward_projections[exp_idx] = forward_proj.astype(np.float32)
    
    # Clean up
    del reconstruction, spatial_projection, forward_proj

print(f"\nForward projections shape: {all_forward_projections.shape}")
print(f"Expected shape: ({num_experiments}, {num_angles}, {proj_shape[0]}, {proj_shape[1]}, {num_detector_angles})")

In [ ]:
# Compute standard deviation across experiments for each angle
print("Computing variability across experiments for each projection angle...")

# Compute std across experiments (axis 0) for each angle
# Result shape: (num_angles, proj_shape[0], proj_shape[1], num_detector_angles)
std_per_angle = np.std(all_forward_projections, axis=0)

# Compute total variability per angle (sum of variance across all pixels and detector angles)
variability_per_angle = np.sum(std_per_angle**2, axis=(1, 2, 3))

print(f"\nStandard deviation per angle shape: {std_per_angle.shape}")
print(f"Variability per angle shape: {variability_per_angle.shape}")
print(f"\nVariability statistics:")
print(f"  Mean: {np.mean(variability_per_angle):.4f}")
print(f"  Median: {np.median(variability_per_angle):.4f}")
print(f"  Min: {np.min(variability_per_angle):.4f}")
print(f"  Max: {np.max(variability_per_angle):.4f}")

In [ ]:
# Find angles with highest and lowest variability
num_top_angles = 20

# Get indices sorted by variability
sorted_indices = np.argsort(variability_per_angle)[::-1]  # Descending order

# Top 20 most variable angles
top_variable_indices = sorted_indices[:num_top_angles]
top_variable_values = variability_per_angle[top_variable_indices]

# Bottom 20 least variable angles
least_variable_indices = sorted_indices[-num_top_angles:]
least_variable_values = variability_per_angle[least_variable_indices]

fixed_sparse_values = variability_per_angle[fixed_sparse_indices]

print(f"Top {num_top_angles} most variable angles:")
for i, (idx, val) in enumerate(zip(top_variable_indices, top_variable_values)):
    print(f"  {i+1}. Angle {idx}: variability = {val:.4f}")

print(f"\nTop {num_top_angles} least variable angles:")
for i, (idx, val) in enumerate(zip(least_variable_indices, least_variable_values)):
    print(f"  {i+1}. Angle {idx}: variability = {val:.4f}")

In [ ]:
# Plot variability as a function of angle index
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(variability_per_angle, 'o-', linewidth=1, markersize=3, alpha=0.7)
ax.scatter(top_variable_indices, top_variable_values, color='red', s=50, 
           label=f'Top {num_top_angles} most variable', zorder=5)
ax.scatter(least_variable_indices, least_variable_values, color='green', s=50,
           label=f'Top {num_top_angles} least variable', zorder=5)

ax.scatter(fixed_sparse_indices, fixed_sparse_values, color='blue', s=50,
           label=f'Fixed Sparse Angles', zorder=5)

ax.set_xlabel('Projection Angle Index', fontsize=12)
ax.set_ylabel('Total Variability (Sum of Variance)', fontsize=12)
ax.set_title('Variability Across Projection Angles', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3D Visualization of High-Variability Angles

Visualize the projection angles with the highest variability on a 3D sphere.

In [ ]:
# Extract rotation and tilt angles from geometry
rotation_angles_rad = dc_full.geometry.inner_angles[:]
tilt_angles_rad = list(np.pi/2 - np.array(dc_full.geometry.outer_angles))[:]

# Convert to Cartesian coordinates on unit sphere
r = 1
x_all = r * np.sin(tilt_angles_rad) * np.cos(rotation_angles_rad)
y_all = r * np.sin(tilt_angles_rad) * np.sin(rotation_angles_rad)
z_all = r * np.cos(tilt_angles_rad)

# Extract coordinates for high-variability angles
x_high_var = x_all[top_variable_indices]
y_high_var = y_all[top_variable_indices]
z_high_var = z_all[top_variable_indices]

print(f"Plotted {len(x_all)} total angles")
print(f"Highlighted {len(x_high_var)} high-variability angles in red")

In [ ]:
# Create 3D sphere plot showing all angles and highlighting high-variability ones
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Create sphere surface
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
X = r * np.outer(np.cos(u), np.sin(v))
Y = r * np.outer(np.sin(u), np.sin(v))
Z = r * np.outer(np.ones(np.size(u)), np.cos(v))

# Plot sphere (semi-transparent)
ax.plot_surface(X, Y, Z, color='lightblue', alpha=0.1)

# Plot all projection angles (gray)
ax.scatter(x_all, y_all, z_all, color='gray', s=20, alpha=0.5, label='All angles')

# Highlight high-variability angles (red)
ax.scatter(x_high_var, y_high_var, z_high_var, color='red', s=100, 
           alpha=0.9, edgecolors='darkred', linewidths=2,
           label=f'Top {num_top_angles} highest variability')

ax.set_xlabel('X', fontsize=10)
ax.set_ylabel('Y', fontsize=10)
ax.set_zlabel('Z', fontsize=10)
ax.set_title('Projection Angles: High-Variability Highlighted', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.view_init(elev=10, azim=70)

plt.tight_layout()
plt.show()

In [ ]:
# Compare fixed sparse subset vs high-variability angles in 3D
# Extract coordinates for fixed sparse subset angles
x_sparse = x_all[fixed_sparse_indices]
y_sparse = y_all[fixed_sparse_indices]
z_sparse = z_all[fixed_sparse_indices]

print(f"Fixed sparse subset: {len(fixed_sparse_indices)} angles")
print(f"Top high-variability: {len(top_variable_indices)} angles")

# Check overlap between fixed sparse subset and high-variability angles
overlap = np.intersect1d(fixed_sparse_indices, top_variable_indices)
print(f"Overlap between fixed sparse subset and top {num_top_angles} high-variability: {len(overlap)} angles")
print(f"Overlap indices: {overlap}")

# Create 3D sphere plot
fig = plt.figure(figsize=(14, 12))
ax = fig.add_subplot(111, projection='3d')

# Plot sphere surface (semi-transparent)
ax.plot_surface(X, Y, Z, color='lightblue', alpha=0.05)

# Plot all projection angles (light gray, small)
ax.scatter(x_all, y_all, z_all, color='lightgray', s=15, alpha=0.3, label='All angles')

# Plot fixed sparse subset (blue)
ax.scatter(x_sparse, y_sparse, z_sparse, color='blue', s=80, 
           alpha=0.7, edgecolors='darkblue', linewidths=1.5,
           label=f'Fixed sparse subset ({len(fixed_sparse_indices)})')

# Plot high-variability angles (red) - these will appear on top
ax.scatter(x_high_var, y_high_var, z_high_var, color='red', s=120, 
           alpha=0.9, edgecolors='darkred', linewidths=2,
           label=f'Top {num_top_angles} highest variability', marker='^')

# Highlight overlap if any (yellow stars)
if len(overlap) > 0:
    x_overlap = x_all[overlap]
    y_overlap = y_all[overlap]
    z_overlap = z_all[overlap]
    ax.scatter(x_overlap, y_overlap, z_overlap, color='yellow', s=150,
               alpha=1.0, edgecolors='orange', linewidths=2,
               label=f'Overlap ({len(overlap)})', marker='*', zorder=10)

ax.set_xlabel('X', fontsize=11)
ax.set_ylabel('Y', fontsize=11)
ax.set_zlabel('Z', fontsize=11)
ax.set_title(f'Projection Angles: Fixed Sparse Subset vs High-Variability Angles', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10, loc='upper left')
ax.view_init(elev=10, azim=70)

plt.tight_layout()
plt.show()

In [ ]:
# Create another view focusing only on high-variability angles
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Create sphere surface
ax.plot_surface(X, Y, Z, color='lightgray', alpha=0.1)

# Plot only high-variability angles with color scale based on variability
scatter = ax.scatter(x_high_var, y_high_var, z_high_var, 
                     c=top_variable_values, s=200, cmap='hot',
                     alpha=0.9, edgecolors='black', linewidths=1)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax, shrink=0.6, pad=0.1)
cbar.set_label('Variability', fontsize=12)

ax.set_xlabel('X', fontsize=10)
ax.set_ylabel('Y', fontsize=10)
ax.set_zlabel('Z', fontsize=10)
ax.set_title(f'Top {num_top_angles} Highest-Variability Angles\n(Color-coded by variability)', 
             fontsize=14, fontweight='bold')
ax.view_init(elev=10, azim=70)

plt.tight_layout()
plt.show()

### Compare Projections at Most and Least Variable Angles

Visualize the actual projection images at angles with highest and lowest variability.

In [ ]:
# Select a few angles to visualize in detail
num_angles_to_show = 3
detector_angle_idx = 0  # Show one detector angle

# Most variable angles
high_var_angles = top_variable_indices[:num_angles_to_show]
high_var_labels = [f"Angle {idx}\nVar={variability_per_angle[idx]:.2f}" 
                   for idx in high_var_angles]

# Least variable angles  
low_var_angles = least_variable_indices[:num_angles_to_show]
low_var_labels = [f"Angle {idx}\nVar={variability_per_angle[idx]:.2f}" 
                  for idx in low_var_angles]

print(f"Visualizing detector angle index {detector_angle_idx}")
print(f"\nMost variable angles: {high_var_angles}")
print(f"Least variable angles: {low_var_angles}")

In [ ]:
# Plot standard deviation projections for high-variability angles
fig, axes = plt.subplots(1, num_angles_to_show, figsize=(18, 6))

for i, (angle_idx, label) in enumerate(zip(high_var_angles, high_var_labels)):
    std_proj = std_per_angle[angle_idx, :, :, detector_angle_idx]
    
    im = axes[i].imshow(std_proj.T, cmap='hot', origin='lower')
    axes[i].set_title(f'High Variability\n{label}', fontsize=11, fontweight='bold')
    axes[i].axis('off')
    plt.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)

fig.suptitle(f'Standard Deviation of Projections - Most Variable Angles (Detector angle {detector_angle_idx})',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Plot standard deviation projections for low-variability angles
fig, axes = plt.subplots(1, num_angles_to_show, figsize=(18, 6))

for i, (angle_idx, label) in enumerate(zip(low_var_angles, low_var_labels)):
    std_proj = std_per_angle[angle_idx, :, :, detector_angle_idx]
    
    im = axes[i].imshow(std_proj.T, cmap='hot', origin='lower')
    axes[i].set_title(f'Low Variability\n{label}', fontsize=11, fontweight='bold')
    axes[i].axis('off')
    plt.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)

fig.suptitle(f'Standard Deviation of Projections - Least Variable Angles (Detector angle {detector_angle_idx})',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Compare mean projections from experiments vs ground truth for highest variability angle
most_variable_angle = top_variable_indices[0]

# Compute mean projection across experiments
mean_projection = np.mean(all_forward_projections[:, most_variable_angle, :, :, detector_angle_idx], axis=0)

# Get ground truth projection (need to project ground truth)
print(f"Computing ground truth projection for angle {most_variable_angle}...")
gt_spatial_proj = projector_full.forward(ground_truth.astype(np.float64))
gt_forward_proj = basis_set.forward(gt_spatial_proj)
gt_projection = gt_forward_proj[most_variable_angle, :, :, detector_angle_idx]

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

vmin = min(mean_projection.min(), gt_projection.min())
vmax = max(mean_projection.max(), gt_projection.max())

# Mean projection
im0 = axes[0].imshow(mean_projection.T, cmap='viridis', vmin=vmin, vmax=vmax, origin='lower')
axes[0].set_title(f'Mean Projection\nAngle {most_variable_angle}', fontsize=12, fontweight='bold')
axes[0].axis('off')
plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

# Ground truth projection
im1 = axes[1].imshow(gt_projection.T, cmap='viridis', vmin=vmin, vmax=vmax, origin='lower')
axes[1].set_title(f'Ground Truth Projection\nAngle {most_variable_angle}', fontsize=12, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

# Difference
diff = mean_projection - gt_projection
im2 = axes[2].imshow(diff.T, cmap='RdBu_r', origin='lower',
                     vmin=-np.abs(diff).max(), vmax=np.abs(diff).max())
axes[2].set_title('Difference\n(Mean - Ground Truth)', fontsize=12, fontweight='bold')
axes[2].axis('off')
plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)

fig.suptitle(f'Most Variable Angle (#{most_variable_angle}, variability={variability_per_angle[most_variable_angle]:.2f})',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\nMSE between mean and ground truth projection: {np.mean((mean_projection - gt_projection)**2):.6f}")